In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110680

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,55169.7028
2018-02-28,40520.5553
2018-03-31,47907.8113
2018-04-30,66748.8023
2018-05-31,57526.1507
2018-06-30,62726.4690
2018-07-31,68325.1862
2018-08-31,72650.3577
2018-09-30,53292.0730


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     55169.7028
 2018-02-28     40520.5553
 2018-03-31     47907.8113
 2018-04-30     66748.8023
 2018-05-31     57526.1507
 2018-06-30     62726.4690
 2018-07-31     68325.1862
 2018-08-31     72650.3577
 2018-09-30     53292.0730
 2018-10-31     47242.4218
 2018-11-30     51874.8727
 2018-12-31     46514.6613
 2019-01-31     75285.0281
 2019-02-28     69417.9414
 2019-03-31     84854.4747
 2019-04-30     83852.5262
 2019-05-31     83733.6071
 2019-06-30     86497.1106
 2019-07-31    101502.0485
 2019-08-31    111472.9922
 2019-09-30     88197.5820
 2019-10-31     84040.6131
 2019-11-30     70117.4773
 2019-12-31     85624.7247
 2020-01-31     78374.0824
 2020-02-29     45082.4159
 2020-03-31     59299.5897
 2020-04-30     44676.5935
 2020-05-31     65710.0208
 2020-06-30     61470.8971
 2020-07-31     74460.2382
 2020-08-31     78582.1890
 2020-09-30     75118.5205
 2020-10-31     64801.0731
 2020-11-30     45782.6634
 2020-12-31     57565.5098

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.854594
p-value : 0.050889
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -8.305723
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=936.435, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=925.156, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=920.534, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=919.830, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=920.750, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=919.170, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=914.429, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=916.302, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=917.198, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=919.208, Time=0.06 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=921.214, Time=0.10 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=915.945, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=917.639, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=919.190, Time=0.11 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=920.695, Time=0.20 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -454.213
Method:                       css-mle   S.D. of innovations          11956.160
Date:                Sat, 11 Sep 2021   AIC                            914.427
Time:                        04:21:36   BIC                            919.640
Sample:                             0   HQIC                           916.337
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.663e+04   5119.712     13.014      0.000    5.66e+04    7.67e+04
ar.L1.y        0.6547      0.113      5.809      0.000       0.434       0.876
                                    Roots           

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([66535.51476066, 66566.8703052 ]), array([11956.1598643 , 14290.45395847]), array([[43101.87203323, 89969.15748809],
       [38558.09522387, 94575.64538654]]))
